In [1]:
import torch
import torch.nn as nn
from time import time
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

In [4]:
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

In [5]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("In Model: in:", input.size(),
              "out:", output.size())

        return output

In [6]:
#DataParallel 
# splits your data automatically and sends 
# job orders to multiple models on several GPUs. After each 
# model finishes their job, DataParallel collects and merges 
# the results before returning it to you.

In [7]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

model.to(device)

Let's use 1 GPUs!


DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [8]:
ini = time()
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("\tOutside: in:", input.size(),
          "out:", output.size())
print(time()-ini)

In Model: in: torch.Size([30, 5]) out: torch.Size([30, 2])
	Outside: in: torch.Size([30, 5]) out: torch.Size([30, 2])
In Model: in: torch.Size([30, 5]) out: torch.Size([30, 2])
	Outside: in: torch.Size([30, 5]) out: torch.Size([30, 2])
In Model: in: torch.Size([30, 5]) out: torch.Size([30, 2])
	Outside: in: torch.Size([30, 5]) out: torch.Size([30, 2])
In Model: in: torch.Size([10, 5]) out: torch.Size([10, 2])
	Outside: in: torch.Size([10, 5]) out: torch.Size([10, 2])
0.517202615737915
